In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [12]:
from dotenv import load_dotenv

load_dotenv()
KEY=os.getenv("API_KEY")

In [13]:
print(KEY)

sk-78db4ad7682144269cb3faf3383d1fe8


In [84]:
llm=ChatOpenAI(api_key=KEY, base_url="https://api.deepseek.com"
               ,model_name='deepseek-chat', temperature=0.7)

In [85]:
reply=llm.predict("Which GPT LLM model do you internally use?")

In [86]:
print(reply)

 I am an intelligent assistant developed by China's DeepSeek company, named DeepSeek Chat. My internal technology is based on the DeepSeek large language model, not the GPT model from OpenAI. I am designed to provide information inquiry, dialogue exchange, and problem-solving services.


In [87]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [88]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [89]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [90]:
mcq_generation_template=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE 
)

In [91]:
mcq_chain=LLMChain(llm=llm,prompt=mcq_generation_template, output_key='mcq', verbose=True)

In [92]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{mcq}

Check from an expert English Writer of the above quiz:
"""

In [93]:
mcq_evalution_prompt=PromptTemplate(
    input_variables=['subject','mcq'],
    template=TEMPLATE2
    )

In [94]:
review_chain=LLMChain(llm=llm,prompt=mcq_evalution_prompt,output_key='review', verbose=True)

In [95]:
generate_evaluate_chain=SequentialChain(chains=[mcq_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["mcq", "review"], verbose=True,)

In [96]:
file_path=r"/Users/mohammedfarhan/Desktop/Pacewisdom/mcq-Gen/ai_mcq_gen/data.txt"

In [97]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [98]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [117]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":5,
            "subject":"Java Programming Language",
            "tone":"Simple",
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Java is a high-level, class-based, object-oriented programming language that is designed to have as few implementation dependencies as possible. It is a general-purpose programming language intended to let programmers write once, run anywhere (WORA),[16] meaning that compiled Java code can run on all platforms that support Java without the need to recompile.[17] Java applications are typically compiled to bytecode that can run on any Java virtual machine (JVM) regardless of the underlying computer architecture. The syntax of Java is similar to C and C++, but has fewer low-level facilities than either of them. The Java runtime provides dynamic capabilities (such as reflection and runtime code modification) that are typically not available in traditional compiled languages.

Java gained popularity shortly after its release, and has been a very popular programming language since the

In [118]:
print(f"Total Tokens : {cb.total_tokens}")
print(f"Prompt Tokens : {cb.prompt_tokens}")
print(f"Completion Tokens : {cb.completion_tokens}")
print(f"Total Cost : ${cb.total_cost}")

Total Tokens : 1818
Prompt Tokens : 1274
Completion Tokens : 544
Total Cost : $0.0


In [119]:
response

{'text': "Java is a high-level, class-based, object-oriented programming language that is designed to have as few implementation dependencies as possible. It is a general-purpose programming language intended to let programmers write once, run anywhere (WORA),[16] meaning that compiled Java code can run on all platforms that support Java without the need to recompile.[17] Java applications are typically compiled to bytecode that can run on any Java virtual machine (JVM) regardless of the underlying computer architecture. The syntax of Java is similar to C and C++, but has fewer low-level facilities than either of them. The Java runtime provides dynamic capabilities (such as reflection and runtime code modification) that are typically not available in traditional compiled languages.\n\nJava gained popularity shortly after its release, and has been a very popular programming language since then.[18] Java was the third most popular programming language in 2022 according to GitHub.[19] Alt

In [120]:
mcqs=response.get('mcq')

In [121]:
mcqs_string=mcqs.strip('` \n\t').replace('json','')

In [124]:
mcqs_json=json.loads(mcqs_string)

In [125]:


quiz_table_data = []
for key, value in mcqs_json.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [127]:
quiz=pd.DataFrame(quiz_table_data)

In [128]:
quiz.to_csv("java.csv",index=False)